In [6]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import swifter

In [5]:
file = '311_Service_Requests_from_2010_to_Present.csv'

In [16]:
#print(pd.read_csv(file, nrows=5))
test = pd.read_csv(file, nrows=5)
test = test.rename(columns={c: c.replace(' ', '') for c in test.columns}) 
test = test.rename(columns={c: c.replace('(', '') for c in test.columns})
test = test.rename(columns={c: c.replace(')', '') for c in test.columns})


In [17]:
test.columns = [c.lower() for c in test.columns]
test

,uniquekey,createddate,closeddate,agency,agencyname,complainttype,descriptor,locationtype,incidentzip,incidentaddress,...,vehicletype,taxicompanyborough,taxipickuplocation,bridgehighwayname,bridgehighwaydirection,roadramp,bridgehighwaysegment,latitude,longitude,location
0,24811305,01/18/2013 12:00:00 AM,02/11/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PLUMBING,BASIN/SINK,RESIDENTIAL BUILDING,11219,1454 43 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636336,-73.985246,"(40.63633580546793, -73.98524566850578)"
1,24811306,01/18/2013 12:00:00 AM,01/23/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,FLOOR,RESIDENTIAL BUILDING,10468,2420 DAVIDSON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.861985,-73.902833,"(40.86198534161541, -73.90283327659593)"
2,24811307,01/18/2013 12:00:00 AM,01/18/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10025,858 WEST END AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.798897,-73.970257,"(40.79889676055833, -73.97025674483577)"
3,24811308,01/18/2013 12:00:00 AM,01/23/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PLUMBING,BASIN/SINK,RESIDENTIAL BUILDING,11206,160 VERNON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.695054,-73.945363,"(40.69505379327949, -73.94536269567737)"
4,24811309,01/18/2013 12:00:00 AM,01/23/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,11233,841 HALSEY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.685352,-73.919075,"(40.68535201436509, -73.91907456016442)"


In [18]:
test.columns

Index(['uniquekey', 'createddate', 'closeddate', 'agency', 'agencyname',
       'complainttype', 'descriptor', 'locationtype', 'incidentzip',
       'incidentaddress', 'streetname', 'crossstreet1', 'crossstreet2',
       'intersectionstreet1', 'intersectionstreet2', 'addresstype', 'city',
       'landmark', 'facilitytype', 'status', 'duedate',
       'resolutiondescription', 'resolutionactionupdateddate',
       'communityboard', 'bbl', 'borough', 'xcoordinatestateplane',
       'ycoordinatestateplane', 'opendatachanneltype', 'parkfacilityname',
       'parkborough', 'vehicletype', 'taxicompanyborough',
       'taxipickuplocation', 'bridgehighwayname', 'bridgehighwaydirection',
       'roadramp', 'bridgehighwaysegment', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [6]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'Health_Inspection'
username = 'postgres' # change this to your username

In [7]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://postgres@localhost/Health_Inspection


In [24]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


# Get data into postgres

In [27]:
chunksize = 100000
i = 0
j = 1
#colnames = ['uniquekey', 'createddate', 'closeddate', 'agency', 'agencyname','complainttype', 'descriptor', 'locationtype', 'incidentzip','incidentaddress', 'streetname', 'crossstreet1', 'crossstreet2','intersectionstreet1', 'intersectionstreet2', 'addresstype', 'city','landmark', 'facilitytype', 'status', 'duedate','resolutiondescription', 'resolutionactionupdateddate','communityboard', 'bbl', 'borough', 'xcoordinatestateplane','ycoordinatestateplane', 'opendatachanneltype', 'parkfacilityname','parkborough', 'vehicletype', 'taxicompanyborough','taxipickuplocation', 'bridgehighwayname', 'bridgehighwaydirection','roadramp', 'bridgehighwaysegment', 'latitude', 'longitude','location']
for df in pd.read_csv(file, chunksize=chunksize, iterator=True, low_memory=False):
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
    df = df.rename(columns={c: c.replace('(', '') for c in df.columns})
    df = df.rename(columns={c: c.replace(')', '') for c in df.columns})
    df.columns = [c.lower() for c in df.columns]
    df.index += j
    i+=1
    df.to_sql('ny311table2', engine, if_exists='append')
    j = df.index[-1] + 1

# Use SQL within python

In [28]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)


In [36]:
# query:
sql_query = """
SELECT createddate, complainttype, descriptor,locationtype, incidentzip, landmark, latitude,longitude,(SUBSTR(createddate, 7, 4) || '-' || LEFT(createddate, 2) || '-' || SUBSTR(createddate, 4, 2))::timestamp AS cleaned_date
FROM ny311table2 
WHERE locationtype IN ('Club/Bar/Restaurant','Restaurant/Bar/Deli/Bakery','Restaurant','Food Cart Vendor', 'Catering Service','Cafeteria- College/University','Cafeteria-Public School', 'Cafeteria- Private School','Cafeteria- Private', 'Cafeteria', 'Catering Hall','Bar/Restaurant')
;
"""
health_data_from_sql = pd.read_sql_query(sql_query,con)
health_data_from_sql.head()
len(health_data_from_sql)

236692

In [37]:
test = health_data_from_sql[health_data_from_sql['cleaned_date']>= '2014-01-01']

In [39]:
test.to_csv(r'NY311_part2.csv')

In [38]:
len(test)

140042

# Put week 2 dataset into SQL

In [6]:
file2 = 'DataforWk2.csv'

In [7]:
cd '/Users/rabarry/Documents/Insight/NYC data'

/Users/rabarry/Documents/Insight/NYC data


In [8]:
# Set your postgres username
dbname = 'Health_Inspection'
username = 'postgres' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://postgres@localhost/Health_Inspection


In [9]:
df = pd.read_csv(file2)

In [ ]:
#df.to_sql('DataforWk2', engine, if_exists='append')

In [10]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [44]:
rest_name = 'starbucks'
zip_code = int('10001')

In [45]:
# query:
query = "SELECT \"DBA\", ""\"BORO_cat\",\"CUISINE DESCRIPTION_cat\",\"Second_Last_Insp_Num_CriticialFlags\",\"Second_Last_Insp_Type_cat\",\"Insp_Date_Diff\", \"num_years_active\"  FROM \"DataforWk2\" WHERE \"DBA\" ILIKE '%s' AND \"ZIPCODE\"=%d" %(rest_name, zip_code)
print(query)
query_results=pd.read_sql_query(query,con)
print(query_results)
births = []

SELECT "DBA", "BORO_cat","CUISINE DESCRIPTION_cat","Second_Last_Insp_Num_CriticialFlags","Second_Last_Insp_Type_cat","Insp_Date_Diff", "num_years_active"  FROM "DataforWk2" WHERE "DBA" ILIKE 'starbucks' AND "ZIPCODE"=10001
          DBA  BORO_cat  CUISINE DESCRIPTION_cat  \
0   STARBUCKS         2                       13   
1   STARBUCKS         2                       13   
2   STARBUCKS         2                       13   
3   STARBUCKS         2                       13   
4   STARBUCKS         2                       13   
5   STARBUCKS         2                       13   
6   STARBUCKS         2                       13   
7   STARBUCKS         2                       13   
8   STARBUCKS         2                       13   
9   STARBUCKS         2                       13   
10  STARBUCKS         2                        2   
11  STARBUCKS         2                       13   

    Second_Last_Insp_Num_CriticialFlags  Second_Last_Insp_Type_cat  \
0                             

In [39]:
query_results

,BORO_cat,CUISINE DESCRIPTION_cat,Second_Last_Insp_Num_CriticialFlags,Second_Last_Insp_Type_cat,Insp_Date_Diff,num_years_active
0,2,44,1.0,0,386.0,2.0


In [29]:
for i in range(0,query_results.shape[0]):
    births.append(dict(index=query_results.iloc[i]['index'], attendant=query_results.iloc[i]['attendant'], birth_month=query_results.iloc[i]['birth_month']))
    the_result = ModelIt(patient,births)
    return render_template("output.html", births = births, the_result = the_result)

1

In [30]:
health_data_from_sql

,BORO_cat,CUISINE DESCRIPTION_cat,Second_Last_Insp_Num_CriticialFlags,Second_Last_Insp_Type_cat,Insp_Date_Diff,num_years_active
0,2,44,1.0,0,386.0,2.0


In [40]:
df.iloc[7]

Unnamed: 0                                                 7
CAMIS                                               40357217
Second_Last_Insp_Type                                Initial
Second_Last_Insp_Date                             2018-05-31
Second_Last_Insp_Num_CriticialFlags                        0
Last_Insp_Type                                       Initial
Last_Insp_Date                                    2019-05-22
Last_Insp_Num_CriticialFlags                               1
Insp_Date_Diff                                           356
DBA                                                WILD ASIA
BORO                                                   Bronx
ZIPCODE                                                10460
CUISINE DESCRIPTION                                 American
Latitude                                             40.8506
Longitude                                           -73.8825
Community Board                                          206
Council District        

In [17]:
#df.head()
df.loc[21821]
#df.sort_values('Last_Insp_Num_CriticialFlags', ascending = False)

Unnamed: 0                                            22655
CAMIS                                              50086495
Second_Last_Insp_Type                         Re-inspection
Second_Last_Insp_Date                            2019-05-14
Second_Last_Insp_Num_CriticialFlags                       2
Last_Insp_Type                                      Initial
Last_Insp_Date                                   2019-12-26
Last_Insp_Num_CriticialFlags                              9
Insp_Date_Diff                                          226
DBA                                            SHOKUNIN BBQ
BORO                                               Brooklyn
ZIPCODE                                               11219
CUISINE DESCRIPTION                                Barbecue
Latitude                                            40.6333
Longitude                                           -74.006
Community Board                                         312
Council District                        

# Get final dataset into SQL for flask app

In [2]:
data = pd.read_csv('/Users/rabarry/Documents/Insight/NYC data/SafeEats/data/processed/HD_NY311_Hoods.csv')

In [3]:
data=data.drop('Unnamed: 0', axis=1)
data.columns

Index(['CAMIS', 'Second_Last_Insp_Type', 'Second_Last_Insp_Date',
       'Second_Last_Insp_Num_CriticialFlags', 'Last_Insp_Type',
       'Last_Insp_Date', 'Last_Insp_Num_CriticialFlags', 'Insp_Date_Diff',
       'DBA', 'BORO', 'ZIPCODE', 'CUISINE DESCRIPTION', 'Latitude',
       'Longitude', 'Community Board', 'Council District', 'Census Tract',
       'BIN', 'BBL', 'NTA', 'total_critical_flags', 'first_inspection_date',
       'last_inspection_date', 'num_years_active',
       'avg_num_critical_flags_per_year', 'location', 'is_good_overall',
       'is_good_lastInsp', 'ny311_number_of_complaints_6mo',
       'ny311_number_of_complaints_12mo', 'ny311_number_of_complaints_3mo',
       'Cuisine_Reduced', 'Unnamed: 0.1', 'District Name', 'zip',
       'Income diversity ratio', 'Median household income (2018$)',
       'Population', 'Population density (1,000 persons per square mile)',
       'Poverty rate', 'Public housing (% of rental units)',
       'Racial diversity index', 'Serious cr

In [4]:
conditions = [
    (data['CUISINE DESCRIPTION'] == 'American') |  (data['CUISINE DESCRIPTION'] == 'Hamburgers') | (data['CUISINE DESCRIPTION'] == 'Steak') | (data['CUISINE DESCRIPTION'] == 'Soul Food') | (data['CUISINE DESCRIPTION'] == 'Barbecue') | (data['CUISINE DESCRIPTION'] == 'Hotdogs/Pretzels') | (data['CUISINE DESCRIPTION'] == 'Hotdogs') | (data['CUISINE DESCRIPTION'] == 'Californian') | (data['CUISINE DESCRIPTION'] == 'Southwestern') | (data['CUISINE DESCRIPTION'] == 'Chicken'),
    (data['CUISINE DESCRIPTION'] == 'Chinese') | (data['CUISINE DESCRIPTION'] == 'Japanese') | (data['CUISINE DESCRIPTION'] == 'Asian') | (data['CUISINE DESCRIPTION'] == 'Thai') | (data['CUISINE DESCRIPTION'] == 'Korean') | (data['CUISINE DESCRIPTION'] == 'Vietnamese/Cambodian/Malaysia') | (data['CUISINE DESCRIPTION'] == 'Chinese/Japanese') | (data['CUISINE DESCRIPTION'] == 'Filipino') | (data['CUISINE DESCRIPTION'] == 'Indonesian'), 
    (data['CUISINE DESCRIPTION'] == 'Café/Coffee/Tea') | (data['CUISINE DESCRIPTION'] == 'Bakery') | (data['CUISINE DESCRIPTION'] == 'Donuts') | (data['CUISINE DESCRIPTION'] == 'Juice, Smoothies, Fruit Salads') | (data['CUISINE DESCRIPTION'] == 'Ice Cream, Gelato, Yogurt, Ices') | (data['CUISINE DESCRIPTION'] == 'Bottled beverages, including water, sodas, juices, etc. ') | (data['CUISINE DESCRIPTION'] == 'Pancakes/Waffles')| (data['CUISINE DESCRIPTION'] == 'Nuts/Confectionary'),
    (data['CUISINE DESCRIPTION'] == 'Pizza') | (data['CUISINE DESCRIPTION'] == 'Italian') | (data['CUISINE DESCRIPTION'] == 'Pizza/Italian'),
    (data['CUISINE DESCRIPTION'] == 'Mexican') | (data['CUISINE DESCRIPTION'] == 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)') | (data['CUISINE DESCRIPTION'] == 'Spanish') | (data['CUISINE DESCRIPTION'] == 'Tex-Mex') | (data['CUISINE DESCRIPTION'] == 'Peruvian') | (data['CUISINE DESCRIPTION'] == 'Tapas') | (data['CUISINE DESCRIPTION'] == 'Chilean')      
]
choices = ['American', 'Asian', 'Cafe/Dessert', 'Italian', 'Latin']
data['Cuisine_Reduced'] = np.select(conditions, choices, default='Other')

In [5]:
data['Last_Insp_Type_CAT']=data['Last_Insp_Type'].astype('category')
data['Last_Insp_Type_CAT'] = data['Last_Insp_Type_CAT'].cat.codes
data['Last_Insp_Num_CriticialFlags_CAT']=data['Last_Insp_Num_CriticialFlags'].astype('category')
data['BORO_CAT']=data['BORO'].astype('category')
data['BORO_CAT'] = data['BORO_CAT'].cat.codes
data['num_years_active_CAT']=data['num_years_active'].astype('category')
data['Cuisine_CAT']=data['Cuisine_Reduced'].astype('category')
data['Cuisine_CAT'] = data['Cuisine_CAT'].cat.codes
data['ny311_number_of_complaints_3mo']= data['ny311_number_of_complaints_3mo'].astype('category')

In [6]:
data['Median household income (2018$)'] = data['Median household income (2018$)'].str[1:]
data['Poverty rate'] = data['Poverty rate'].str[:-1]
data['Public housing (% of rental units)'] = data['Public housing (% of rental units)'].str[:-1]

data = data.swifter.apply(lambda x: x.str.strip() if x.dtype =='object' else x)

data = data.swifter.apply(lambda x: x.str.replace(',','') if x.dtype =='object' else x)

data['Median household income (2018$)'] = data['Median household income (2018$)'].astype('float64')

data['Population'] = data['Population'].astype('float64')
data['Poverty rate'] = data['Poverty rate'].astype('float64')
data['Public housing (% of rental units)'] = data['Public housing (% of rental units)'].astype('float64')

In [7]:
data.columns

Index(['CAMIS', 'Second_Last_Insp_Type', 'Second_Last_Insp_Date',
       'Second_Last_Insp_Num_CriticialFlags', 'Last_Insp_Type',
       'Last_Insp_Date', 'Last_Insp_Num_CriticialFlags', 'Insp_Date_Diff',
       'DBA', 'BORO', 'ZIPCODE', 'CUISINE DESCRIPTION', 'Latitude',
       'Longitude', 'Community Board', 'Council District', 'Census Tract',
       'BIN', 'BBL', 'NTA', 'total_critical_flags', 'first_inspection_date',
       'last_inspection_date', 'num_years_active',
       'avg_num_critical_flags_per_year', 'location', 'is_good_overall',
       'is_good_lastInsp', 'ny311_number_of_complaints_6mo',
       'ny311_number_of_complaints_12mo', 'ny311_number_of_complaints_3mo',
       'Cuisine_Reduced', 'Unnamed: 0.1', 'District Name', 'zip',
       'Income diversity ratio', 'Median household income (2018$)',
       'Population', 'Population density (1,000 persons per square mile)',
       'Poverty rate', 'Public housing (% of rental units)',
       'Racial diversity index', 'Serious cr

In [12]:
data4sql = data[['CAMIS','Last_Insp_Date','DBA','ny311_number_of_complaints_3mo',
       'ZIPCODE', 'CUISINE DESCRIPTION', 'Cuisine_Reduced','Latitude',
       'Longitude','Last_Insp_Type_CAT', 'Last_Insp_Num_CriticialFlags_CAT',
       'BORO_CAT', 'num_years_active_CAT', 'Cuisine_CAT', 'avg_num_critical_flags_per_year',
       'Population', 'Population density (1,000 persons per square mile)',
       'Serious housing code violations (per 1,000 privately owned rental units)']]



In [9]:
data['Last_Insp_Date'][0]

'11/4/19'

In [13]:
data4sql['Last_Insp_Date']= pd.to_datetime(data4sql['Last_Insp_Date']).dt.normalize()
data4sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25647 entries, 0 to 25646
Data columns (total 18 columns):
CAMIS                                                                       25647 non-null int64
Last_Insp_Date                                                              25647 non-null datetime64[ns]
DBA                                                                         25647 non-null object
ny311_number_of_complaints_3mo                                              25248 non-null category
ZIPCODE                                                                     25249 non-null float64
CUISINE DESCRIPTION                                                         25647 non-null object
Cuisine_Reduced                                                             25647 non-null object
Latitude                                                                    25248 non-null float64
Longitude                                                                   25248 non-null float

/anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
data4sql.columns = [c.lower() for c in data4sql.columns]
data4sql.columns = [c.strip() for c in data4sql.columns]

data4sql = data4sql.rename(columns={c: c.replace(' ', '_') for c in data4sql.columns}) 
data4sql = data4sql.rename(columns={c: c.replace('(', '') for c in data4sql.columns})
data4sql = data4sql.rename(columns={c: c.replace(')', '') for c in data4sql.columns})
data4sql.columns

Index(['camis', 'last_insp_date', 'dba', 'ny311_number_of_complaints_3mo',
       'zipcode', 'cuisine_description', 'cuisine_reduced', 'latitude',
       'longitude', 'last_insp_type_cat', 'last_insp_num_criticialflags_cat',
       'boro_cat', 'num_years_active_cat', 'cuisine_cat',
       'avg_num_critical_flags_per_year', 'population',
       'population_density_1,000_persons_per_square_mile',
       'serious_housing_code_violations_per_1,000_privately_owned_rental_units'],
      dtype='object')

In [15]:
data4sql.columns = ['camis', 'last_insp_date', 'dba', 'ny311_complaints',
       'zipcode', 'cuisine_description', 'cuisine_reduced','latitude', 'longitude',
       'last_insp_type', 'last_insp_num_flags', 'boro',
       'num_years_active', 'cuisine', 'avg_num_critical_flags_per_year', 'population',
       'population_density','serious_housing_code_violations']
data4sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25647 entries, 0 to 25646
Data columns (total 18 columns):
camis                              25647 non-null int64
last_insp_date                     25647 non-null datetime64[ns]
dba                                25647 non-null object
ny311_complaints                   25248 non-null category
zipcode                            25249 non-null float64
cuisine_description                25647 non-null object
cuisine_reduced                    25647 non-null object
latitude                           25248 non-null float64
longitude                          25248 non-null float64
last_insp_type                     25647 non-null int8
last_insp_num_flags                25647 non-null category
boro                               25647 non-null int8
num_years_active                   25647 non-null category
cuisine                            25647 non-null int8
avg_num_critical_flags_per_year    25647 non-null float64
population               

In [16]:
data4sql.shape
data4sql = data4sql.dropna()
data4sql.shape

(25647, 18)

(25248, 18)

In [17]:
# Set your postgres username
dbname = 'Health_Inspection'
username = 'rabarry' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://rabarry@localhost/Health_Inspection


In [77]:
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import point
data4sql_gdf = gpd.GeoDataFrame(data4sql, geometry = gpd.points_from_xy(data4sql['latitude'], data4sql['longitude']))


In [78]:
data4sql_gdf.head()

,camis,last_insp_date,dba,ny311_complaints,zipcode,cuisine_description,cuisine_reduced,latitude,longitude,last_insp_type,...,income_diversity_ratio,median_income,population,population_density,poverty_rate,public_housing,racial_diversity_index,serious_crime_rate,serious_housing_code_violations,geometry
0,30075445,2019-11-04,MORRIS PARK BAKE SHOP,0.0,10462.0,Bakery,Cafe/Dessert,40.84823,-73.85597,1,...,6.1,40770.0,187975.0,42.5,24.7,39.8,0.59,11.8,78.2,POINT (40.84823 -73.85597)
1,30112340,2019-03-04,WENDY'S,0.0,11225.0,Hamburgers,American,40.66265,-73.96208,0,...,5.3,52050.0,113212.0,66.3,19.1,0.9,0.56,12.4,84.7,POINT (40.66265 -73.96208)
2,30191841,2019-06-06,DJ REYNOLDS PUB AND RESTAURANT,0.0,10019.0,Irish,Other,40.76733,-73.98431,0,...,7.5,103930.0,152455.0,49.6,13.8,0.0,0.59,90.3,8.6,POINT (40.76733 -73.98431)
3,40356018,2019-11-08,RIVIERA CATERERS,0.0,11224.0,American,American,40.57992,-73.98209,0,...,8.2,36810.0,122009.0,35.5,24.8,25.4,0.63,8.4,31.6,POINT (40.57992 -73.98209)
4,40356151,2018-10-06,BRUNOS ON THE BOULEVARD,0.0,11369.0,American,American,40.76397,-73.88097,0,...,4.0,52980.0,146301.0,40.6,15.6,0.0,0.58,7.5,19.2,POINT (40.76397 -73.88097)


In [ ]:
from geoalchemy2 import Geometry
data4sql_gdf.to_sql('dataforapp', engine, if_exists='replace',index=False, dtype={'geometry': Geometry('POINT')})

In [18]:
data4sql.to_sql('dataforapp', engine, if_exists='replace',index=False)

In [19]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [37]:
#rest_name = 'starbucks'
#zip_code = int('10001')
Dist = '3'
location = (40.7601888, -73.98435270226332)
cuisine = 'Asian'

In [38]:
query = "SELECT latitude, longitude, dba,boro, last_insp_type, last_insp_num_flags,ny311_complaints,zipcode, cuisine_description,num_years_active, cuisine, avg_num_critical_flags_per_year, population,population_density, serious_housing_code_violations  FROM dataforapp WHERE cuisine_reduced= '%s' " %(cuisine) 
query_results=pd.read_sql_query(query,con)


In [39]:
query_results.columns

Index(['latitude', 'longitude', 'dba', 'boro', 'last_insp_type',
       'last_insp_num_flags', 'ny311_complaints', 'zipcode',
       'cuisine_description', 'num_years_active', 'cuisine',
       'avg_num_critical_flags_per_year', 'population', 'population_density',
       'serious_housing_code_violations'],
      dtype='object')

In [40]:
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import Point, MultiPoint
import shapely as sh
from geopy.distance import geodesic
origin = Point(location)

In [41]:
def get_miles(row, location):
    loc1 = (row['latitude'], row['longitude'])
    return geodesic(loc1, location).miles

In [42]:
query_results['distance'] = query_results.swifter.apply(lambda row: get_miles(row,location),axis=1)


In [43]:
Dist=3
data = query_results[query_results['distance']< Dist]
data = data.sort_values('distance')

In [194]:
#data4model['insp_date_diff2']= data4model['insp_date_diff'].astype(str)
data['insp_date_diff2'] = (data['insp_date_diff'].astype(str)).swifter.apply(lambda x: x.split('d')[0])

In [44]:
 data4model = data[['avg_num_critical_flags_per_year', 'ny311_complaints',
       'last_insp_type', 'last_insp_num_flags',
       'boro', 'num_years_active', 'cuisine', 
       'population', 'population_density','serious_housing_code_violations']]

In [45]:
cd '/Users/rabarry/Documents/Insight/NYC data/SafeEats/SafeEats_flask'

/Users/rabarry/Documents/Insight/NYC data/SafeEats/SafeEats_flask


In [46]:
import joblib
model_from_pickle = joblib.load('final_model.pkl')
results = model_from_pickle.predict(data4model)
unique_elements, count_elements = np.unique(results, return_counts=True)
print(np.asarray(count_elements))

[1166]


In [47]:
data['result'] = results.tolist()

In [48]:
data = data[data['result']==1]
len(data)

1166

In [212]:
data['zipcode'] = data['zipcode'].astype(int)
data.head()

,latitude,longitude,dba,boro,last_insp_type,last_insp_num_flags,ny311_complaints,zipcode,cuisine_description,num_years_active,...,population_density,poverty_rate,public_housing,racial_diversity_index,serious_crime_rate,serious_housing_code_violations,insp_date_diff,distance,insp_date_diff2,result
3830,40.73661,-73.99068,MACARON PARLOUR,2,0,1,0.0,10003,Nuts/Confectionary,0,...,92.0,27.3,24.0,0.71,14.3,38.0,49 days 19:48:51.112485,1.660536,49,1
3779,40.73661,-73.99068,WAFELS & DINGES,2,0,1,0.0,10003,Pancakes/Waffles,0,...,92.0,27.3,24.0,0.71,14.3,38.0,43 days 19:48:51.112485,1.660536,43,1
3782,40.73661,-73.99068,NO CHEWING ALLOWED!,2,0,0,0.0,10003,Nuts/Confectionary,0,...,92.0,27.3,24.0,0.71,14.3,38.0,49 days 19:48:51.112485,1.660536,49,1
3823,40.73661,-73.99068,THE FLO KISS,2,0,1,0.0,10003,Nuts/Confectionary,0,...,92.0,27.3,24.0,0.71,14.3,38.0,49 days 19:48:51.112485,1.660536,49,1
3764,40.73661,-73.99068,UNITED CHOCOLATE WORKS,2,0,0,0.0,10003,Nuts/Confectionary,0,...,92.0,27.3,24.0,0.71,14.3,38.0,49 days 19:48:51.112485,1.660536,49,1


In [206]:
top3 = {'place1name': data['dba'].iloc[0],
'place1zip': data['zipcode'].iloc[0],
'place2name': data['dba'].iloc[1],
'place2zip': data['zipcode'].iloc[1],
'place3name': data['dba'].iloc[2],
'place3zip': data['zipcode'].iloc[2]}

In [215]:
top3_2 = data[['dba', 'zipcode']][0:3]
top3_2

,dba,zipcode
3830,MACARON PARLOUR,10003
3779,WAFELS & DINGES,10003
3782,NO CHEWING ALLOWED!,10003


In [208]:
print(top3)

{'place1name': 'MACARON PARLOUR', 'place1lat': 40.73661, 'place1long': -73.99068, 'place2name': 'WAFELS & DINGES', 'place2lat': 40.73661, 'place2long': -73.99068, 'place3name': 'NO CHEWING ALLOWED!', 'place3lat': 40.73661, 'place3long': -73.99068}


# Health Inspection Data

In [4]:
cd 'raw data'

/Users/rabarry/Documents/Insight/NYC data/SafeEats/data/raw data


In [7]:
HD = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

In [12]:
# Set your postgres username
dbname = 'Health_Inspection'
username = 'postgres' # change this to your username
engine = create_engine('postgres://%s@localhost:5430/%s'%(username,dbname))
print(engine.url)

postgres://postgres@localhost:5430/Health_Inspection


In [14]:
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [16]:
#HD.to_sql('hd_data', engine, if_exists='replace',index=False)

In [18]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username, port = 5430)

In [ ]:
mask = HD['INSPECTION TYPE'].str.contains(r'Initial', na= True)
HD.loc[mask,'inspection_type_recode']= 'Initial'
mask2 = HD['INSPECTION TYPE'].str.contains(r'Re-inspection', na= True)
HD.loc[mask2,'inspection_type_recode']= 'Re-inspection'
mask3 = HD['INSPECTION TYPE'].str.contains(r'Compliance', na= True)
HD.loc[mask3,'inspection_type_recode']= 'Initial'
mask4 = HD['INSPECTION TYPE'].str.contains(r'Reopening', na= True)
HD.loc[mask4,'inspection_type_recode']= 'Initial'
mask5 = HD['INSPECTION TYPE'].str.contains(r'Limited', na= True)
HD.loc[mask5,'inspection_type_recode']= 'Initial'

In [ ]:
# query:
sql_query = """
ALTER TABLE hd_data
ADD COLUMN flag_recode INT,
ADD COLUMN inspection_type_recode VARCHAR;
"""

with engine.begin() as conn:
    conn.execute(sql_query)

In [39]:
# query:
sql_query = """
UPDATE hd_data
SET flag_recode = CASE WHEN \"CRITICAL FLAG\"= 'Y' THEN 1 ELSE 0 END,
SET inspection_type_recode = CASE WHEN \"INSPECTION TYPE\" ILIKE '%initial%' OR \"INSPECTION TYPE\" ILIKE '%compliance%' OR \"INSPECTION TYPE\" ILIKE '%reopening%' OR \"INSPECTION TYPE\" ILIKE '%limited%' THEN 'Initial' WHEN \"INSPECTION TYPE\" ILIKE '%re-inspection%' THEN 'Re-inspection' END;
"""
health_data_from_sql = pd.read_sql_query(sql_query,con)
health_data_from_sql.head()

DatabaseError: Execution failed on sql '
UPDATE hd_data
SET flag_recode = CASE WHEN "CRITICAL FLAG"= 'Y' THEN 1 ELSE 0 END,
SET inspection_type_recode = CASE WHEN "INSPECTION TYPE" ILIKE '%initial%' OR "INSPECTION TYPE" ILIKE '%compliance%' OR "INSPECTION TYPE" ILIKE '%reopening%' OR "INSPECTION TYPE" ILIKE '%limited%' THEN 'Initial' WHEN "INSPECTION TYPE" ILIKE '%re-inspection%' THEN 'Re-inspection' END;
': syntax error at or near "inspection_type_recode"
LINE 4: SET inspection_type_recode = CASE WHEN "INSPECTION TYPE" ILI...
            ^


In [33]:
# query:
sql_query = """
SELECT \"CAMIS\", \"DBA\", \"BORO\", \"CUISINE DESCRIPTION\", \"INSPECTION DATE\", 
    CASE WHEN \"CRITICAL FLAG\"= 'Y' THEN 1
        ELSE 0
    END AS flag_recode,
    CASE WHEN \"INSPECTION TYPE\" ILIKE '%initial%' OR \"INSPECTION TYPE\" ILIKE '%compliance%' OR \"INSPECTION TYPE\" ILIKE '%reopening%' OR \"INSPECTION TYPE\" ILIKE '%limited%' THEN 'Initial'
        WHEN \"INSPECTION TYPE\" ILIKE '%re-inspection%' THEN 'Re-inspection'
    END AS inspection_type_recode
    
FROM hd_data 
GROUP BY \"CAMIS\", \"INSPECTION DATE\";
"""
health_data_from_sql = pd.read_sql_query(sql_query,con)
health_data_from_sql.head()

,CAMIS,DBA,BORO,CUISINE DESCRIPTION,INSPECTION DATE,flag_recode,inspection_type_recode
0,40918579,PISTICCI RESTAURANT,Manhattan,Italian,11/13/2017,0,Initial
1,50002284,KINANM LOUNGE,Brooklyn,Caribbean,01/18/2018,0,Re-inspection
2,50048386,MAMMA LUCIANA'S,Queens,Pizza/Italian,11/29/2018,1,Re-inspection
3,41647630,KHAN LOUNGE,Queens,Café/Coffee/Tea,10/11/2018,1,Initial
4,40534893,THE GREAT BURRITO MEXICAN KITCHEN,Manhattan,Mexican,10/24/2016,1,Initial
